In [ ]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import shutil
import zipfile
import numpy as np 
import pandas as pd 
import torch.optim as optim
import os

from PIL import Image
from tqdm import tqdm
from torch import nn
from google.colab import drive
from torchvision import transforms, models, datasets
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(0)

In [ ]:
path_to_zip = './drive/MyDrive/breeds.zip'

In [ ]:
!mkdir ./breeds

In [ ]:
with zipfile.ZipFile(path_to_zip,"r") as zip_ref:
    zip_ref.extractall("./breeds")

In [ ]:
mean = [0.4803, 0.4503, 0.3951]
std = [0.2627, 0.2583, 0.2669]

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
cats_dataset=datasets.ImageFolder(root="./breeds/images/Cats",transform=train_transforms)
dogs_dataset=datasets.ImageFolder(root="./breeds/images/Dogs",transform=train_transforms, target_transform=lambda y: y + 12)
full_dataset = torch.utils.data.ConcatDataset([cats_dataset, dogs_dataset])
train_set, test_set = torch.utils.data.random_split(full_dataset, [5000, 1069])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=8)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=8)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.fc = nn.Sequential(torch.nn.Linear(512, 37), nn.Softmax())

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/breed_resnet/resnet18_s2_700.pt'))
model = model.to(device)

In [ ]:
#FOR CPU 
model.load_state_dict(torch.load('./drive/MyDrive/breed_resnet/resnet18_s2_700.pt', map_location=torch.device('cpu')))
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 1000

In [ ]:
model.train()
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device);
        labels = labels.to(device);

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.8f}')
            running_loss = 0.0
    if epoch % 100 == 0: torch.save(model.state_dict(), './drive/MyDrive/breed_resnet/resnet18_s2_' + str(epoch) + '.pt')

print('Finished Training')

In [ ]:
labels_map = {
    0: "Animal: Cat / Breed: Abyssinian",
    1: "Animal: Cat / Breed: Bengal",
    2: "Animal: Cat / Breed: Birman",
    3: "Animal: Cat / Breed: Bombay",
    4: "Animal: Cat / Breed: British Shorthair",
    5: "Animal: Cat / Breed: Egyptian Mau",
    6: "Animal: Cat / Breed: Maine Coon",
    7: "Animal: Cat / Breed: Persian",
    8: "Animal: Cat / Breed: Ragdoll",
    9: "Animal: Cat / Breed: Russian Blue",
    10: "Animal: Cat / Breed: Siamese",
    11: "Animal: Cat / Breed: Sphynx",
    12: "Animal: Dog / Breed: American Bulldog",
    13: "Animal: Dog / Breed: American Pit Bull Terrier",
    14: "Animal: Dog / Breed: Basset Hound",
    15: "Animal: Dog / Breed: Beagle",
    16: "Animal: Dog / Breed: Boxer",
    17: "Animal: Dog / Breed: Chihuahua",
    18: "Animal: Dog / Breed: English Cocker Spaniel",
    19: "Animal: Dog / Breed: English Setter",
    20: "Animal: Dog / Breed: German Shorthaired",
    21: "Animal: Dog / Breed: Great Pyrenees",
    22: "Animal: Dog / Breed: Havanese",
    23: "Animal: Dog / Breed: Japanese Chin",
    24: "Animal: Dog / Breed: Keeshond",
    25: "Animal: Dog / Breed: Leonberger",
    26: "Animal: Dog / Breed: Miniature Pinscher",
    27: "Animal: Dog / Breed: Newfoundland",
    28: "Animal: Dog / Breed: Pomeranian",
    29: "Animal: Dog / Breed: Pug",
    30: "Animal: Dog / Breed: Saint Bernard",
    31: "Animal: Dog / Breed: Samoyed",
    32: "Animal: Dog / Breed: Scottish Terrier",
    33: "Animal: Dog / Breed: Shiba Inu",
    34: "Animal: Dog / Breed: Staffordshire Bull Terrier",
    35: "Animal: Dog / Breed: Wheaten Terrier",
    36: "Animal: Dog / Breed: Yorkshire Terrier"
}


In [ ]:
inv_mean = [-0.4803/0.2627, -0.4503/0.2583, -0.3951/0.2669]
inv_std = [1/0.2627, 1/0.2583, 1/0.2669]
inv_normalize = transforms.Normalize(
    mean=inv_mean,
    std=inv_std
)

In [ ]:
model.eval()
model = model.to(device)
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        fig, ax = plt.subplots()
        images = images.to(device);
        labels = labels.to(device);
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        true_breed = labels_map[labels[0].cpu().detach().numpy().item()]
        predicted_breed = labels_map[predicted[0].cpu().detach().numpy().item()]
        ax.set_title('True: ' + true_breed + ' | Predicted: ' + predicted_breed)
        ax.imshow(inv_normalize(images[0]).permute(1, 2, 0).cpu().detach().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total} %')

In [ ]:
model.eval()
#image = Image.open('./basset_hound_108.jpg')
#image = Image.open('./beagle1.jpeg')
image = Image.open('./img.jpg')


transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.PILToTensor(),
    transforms.Resize((224, 224)),
    #transforms.Normalize(mean, std),
    #transforms.ToTensor(),
])

print(transform(image).shape)
img_tensor = transform(image).float().to(device)
img_tensor = torch.unsqueeze(img_tensor, dim=0)
output = model(img_tensor)
print(output)
_, pred = torch.max(output, 1)
predicted_breed = labels_map[pred[0].cpu().detach().numpy().item()]
predicted_breed